<a href="https://colab.research.google.com/github/maurya-bitlegacy/FastAttack/blob/main/Textfooler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U gensim==4.0.0
!pip install python-Levenshtein
!pip install textattack

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("./drive/MyDrive/models/fasttext.model") # Change this location

In [22]:
model1.wv.most_similar('dog')

[('foxhound', 0.8222606182098389),
 ('hound', 0.8159233927726746),
 ('wolfhound', 0.7827266454696655),
 ('horehound', 0.7729225158691406),
 ('sheepdog', 0.7700775265693665),
 ('bloodhound', 0.7668366432189941),
 ('greyhound', 0.7553040981292725),
 ('hounds', 0.7417516708374023),
 ('bulldog', 0.7402377724647522),
 ('coonhound', 0.7356321215629578)]

# Attacking BERT-base-uncased-yelp-polarity model

In [4]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-yelp-polarity")


model_wrapper = HuggingFaceModelWrapper(model, tokenizer) # Defining our target model

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:09<00:00, 49.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp4nuzx0e3.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


## Framing the attack with our recipe

Defining our transformation

In [5]:
# Defining our transformation
from textattack.transformations import WordSwap

class Swapper(WordSwap): #Fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return a word containing the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]] # If all candidates words contain the input word

### Defining our attack recipe:

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]
# # We'll use the Greedy search method
search_method = GreedyWordSwapWIR("weighted-saliency")
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Using this dataset for the attack

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset yelp_polarity, split test.


Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581. Subsequent calls will reuse this data.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


### Attacking and displaying results:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:04<01:44,  4.35s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:07<01:27,  3.80s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:08<01:00,  2.77s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:09<00:52,  2.48s/it]

...

[Succeeded / Failed / Total] 19 / 4 / 23:  92%|█████████▏| 23/25 [02:07<00:11,  5.56s/it]

[Succeeded / Failed / Total] 20 / 4 / 24:  96%|█████████▌| 24/25 [02:15<00:05,  5.65s/it]

[Succeeded / Failed / Total] 21 / 4 / 25: 100%|██████████| 25/25 [02:17<00:00,  5.49s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 12.51% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 238.84 |
+-------------------------------+--------+


textattack: Attack time: 137.2645184993744s


## Framing the attack using Textfooler attack recipe

In [9]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=f3a5a5a28ad599c3b03862dcd7defb2220f06578196125a1c549d5d3d802cd8c
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


### Declaring Textfoooler's recipe:

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import WordSwapEmbedding

from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren", "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly", "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those", "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless", "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
        )
        # fmt: on
constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
use_constraint = UniversalSentenceEncoder(
            threshold=0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
search_method = GreedyWordSwapWIR(wir_method="delete")

# Now, let's make the attack from the 4 components:
attack1 = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Reusing dataset yelp_polarity (/root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581)
textattack: Loading datasets dataset yelp_polarity, split test.


### Attacking and printing results

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]5 out of the last 5 calls to <function recreate_function.<locals>.restored_function_body at 0x7fe9d9ab3710> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:07<03:11,  7.97s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:11<02:10,  5.67s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:12<01:28,  4.03s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:14<01:14,  3.53s/it]

...

[Succeeded / Failed / Total] 23 / 0 / 23:  92%|█████████▏| 23/25 [03:48<00:19,  9.94s/it]

[Succeeded / Failed / Total] 24 / 0 / 24:  96%|█████████▌| 24/25 [03:55<00:09,  9.81s/it]

[Succeeded / Failed / Total] 25 / 0 / 25: 100%|██████████| 25/25 [03:57<00:00,  9.48s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 25     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 10.89% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 479.08 |
+-------------------------------+--------+


textattack: Attack time: 237.07913446426392s


# Attacking bert-base-uncased-imdb model

In [6]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-imdb")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset

## Attacking using our attack recipe:

In [8]:
# Attacking and printing results
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:06<02:39,  6.65s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:23<04:34, 11.94s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:41<05:02, 13.77s/it]

[Succeeded / Failed / Total] 3 / 1 / 4:  16%|█▌        | 4/25 [00:47<04:08, 11.85s/it]

...

[Succeeded / Failed / Total] 20 / 3 / 25: 100%|██████████| 25/25 [05:20<00:00, 12.81s/it]

[Succeeded / Failed / Total] 20 / 4 / 26: : 26it [05:29, 12.69s/it]

[Succeeded / Failed / Total] 21 / 4 / 27: : 27it [05:42, 12.70s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 14.81% |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 5.29%  |
| Average num. words per input: | 248.96 |
| Avg num queries:              | 419.72 |
+-------------------------------+--------+


textattack: Attack time: 342.94589352607727s


## Attacking using textfooler's recipe

In [11]:
# Attacking and printing results
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:09<03:48,  9.52s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:18<03:32,  9.23s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:27<03:25,  9.33s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:37<03:18,  9.46s/it]

...

[Succeeded / Failed / Total] 23 / 0 / 25: 100%|██████████| 25/25 [06:09<00:00, 14.79s/it]

[Succeeded / Failed / Total] 24 / 0 / 26: : 26it [06:33, 15.13s/it]

[Succeeded / Failed / Total] 25 / 0 / 27: : 27it [06:40, 14.82s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 25     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 7.99%  |
| Average num. words per input: | 248.96 |
| Avg num queries:              | 663.64 |
+-------------------------------+--------+


textattack: Attack time: 400.18313431739807s


# Attacking AlBERT-base-v2-yelp-polarity model

In [23]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

## Attacking using our attack recipe

In [25]:
# Attacking and printing results
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:06<02:34,  6.42s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:10<02:05,  5.45s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:11<01:26,  3.92s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:14<01:14,  3.57s/it]

...

[Succeeded / Failed / Total] 18 / 5 / 23:  92%|█████████▏| 23/25 [03:16<00:17,  8.53s/it]

[Succeeded / Failed / Total] 19 / 5 / 24:  96%|█████████▌| 24/25 [03:27<00:08,  8.65s/it]

[Succeeded / Failed / Total] 19 / 6 / 25: 100%|██████████| 25/25 [03:30<00:00,  8.41s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 76.0%  |
| Average perturbed word %:     | 11.43% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 242.76 |
+-------------------------------+--------+


textattack: Attack time: 210.15210342407227s


## Attacking using Textfooler's recipe

In [15]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:06<02:42,  6.77s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:11<02:13,  5.82s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:12<01:32,  4.21s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:14<01:18,  3.74s/it]

...

[Succeeded / Failed / Total] 22 / 1 / 23:  92%|█████████▏| 23/25 [04:50<00:25, 12.64s/it]

[Succeeded / Failed / Total] 23 / 1 / 24:  96%|█████████▌| 24/25 [05:13<00:13, 13.08s/it]

[Succeeded / Failed / Total] 24 / 1 / 25: 100%|██████████| 25/25 [05:16<00:00, 12.67s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 4.0%   |
| Attack success rate:          | 96.0%  |
| Average perturbed word %:     | 9.11%  |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 480.92 |
+-------------------------------+--------+


textattack: Attack time: 316.8039469718933s


# Attacking AlBERT-base-v2-IMDB model

In [16]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = AutoTokenizer("textattack/albert-base-v2-imdb")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset

## Attacking using our recipe:

In [18]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:07<02:59,  7.46s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:26<05:06, 13.31s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:39<04:53, 13.33s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:46<04:03, 11.58s/it]

...

[Succeeded / Failed / Total] 21 / 3 / 26: : 26it [06:59, 16.13s/it]

[Succeeded / Failed / Total] 21 / 3 / 27: : 27it [06:59, 15.53s/it]

[Succeeded / Failed / Total] 22 / 3 / 28: : 28it [07:06, 15.22s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 22     |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 89.29% |
| Accuracy under attack:        | 10.71% |
| Attack success rate:          | 88.0%  |
| Average perturbed word %:     | 5.2%   |
| Average num. words per input: | 244.71 |
| Avg num queries:              | 434.92 |
+-------------------------------+--------+


textattack: Attack time: 426.0297956466675s


## Attacking using Textfooler's recipe:

In [20]:
# Attacking and printing results
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:08<03:23,  8.47s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:18<03:32,  9.25s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:28<03:26,  9.38s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:33<02:54,  8.29s/it]

...

[Succeeded / Failed / Total] 24 / 0 / 26: : 26it [07:49, 18.08s/it]

[Succeeded / Failed / Total] 24 / 0 / 27: : 27it [07:50, 17.41s/it]

[Succeeded / Failed / Total] 25 / 0 / 28: : 28it [07:53, 16.93s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 25     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 89.29% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 8.35%  |
| Average num. words per input: | 244.71 |
| Avg num queries:              | 674.2  |
+-------------------------------+--------+


textattack: Attack time: 473.955806016922s
